In [5]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from sklearn import datasets, metrics
from sklearn.model_selection import train_test_split

In [6]:
# 1. 데이터 로드 & 전처리
digits = datasets.load_digits()
X_img = digits.images.astype("float32") / 16.0   # 원래 픽셀 범위 0~16 → 0~1 스케일링
y = digits.target
num_classes = len(np.unique(y))

In [7]:
# CNN용 채널 차원 추가, MLP용 평탄화는 모델에서 처리
X_cnn = X_img[..., np.newaxis]  # (n, 8, 8, 1)
y_cat = to_categorical(y, num_classes)

In [8]:
# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(
    X_cnn, y_cat, test_size=0.2, random_state=42, stratify=y
)

In [9]:
# 2. 모델 선택
MODEL = "mlp"  # "mlp" 혹은 "cnn"

if MODEL == "mlp":
    # MLP: 8x8 이미지를 Flatten해서 Dense로 분류
    model = models.Sequential([
        layers.Input(shape=(8, 8, 1)),
        layers.Flatten(),
        layers.Dense(128, activation="relu"),
        layers.Dropout(0.2),
        layers.Dense(64, activation="relu"),
        layers.Dense(num_classes, activation="softmax")
    ])
elif MODEL == "cnn":
    # 작은 CNN: Conv → Pool → Dense
    model = models.Sequential([
        layers.Input(shape=(8, 8, 1)),
        layers.Conv2D(16, kernel_size=3, padding="same", activation="relu"),
        layers.MaxPooling2D(),
        layers.Conv2D(32, kernel_size=3, padding="same", activation="relu"),
        layers.GlobalAveragePooling2D(),
        layers.Dense(32, activation="relu"),
        layers.Dense(num_classes, activation="softmax")
    ])
else:
    raise ValueError("MODEL 은 'mlp' 또는 'cnn' 중 하나여야 합니다.")

In [10]:
# 3. 컴파일 & 학습
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

es = EarlyStopping(monitor="val_accuracy", patience=10, restore_best_weights=True)
history = model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=100,
    batch_size=32,
    callbacks=[es],
    verbose=1
)

Epoch 1/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 0.2434 - loss: 2.1713 - val_accuracy: 0.7674 - val_loss: 1.5747
Epoch 2/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7839 - loss: 1.3543 - val_accuracy: 0.8681 - val_loss: 0.7946
Epoch 3/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8690 - loss: 0.6679 - val_accuracy: 0.8993 - val_loss: 0.4480
Epoch 4/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9012 - loss: 0.4525 - val_accuracy: 0.9201 - val_loss: 0.3386
Epoch 5/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9117 - loss: 0.3577 - val_accuracy: 0.9236 - val_loss: 0.2756
Epoch 6/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9287 - loss: 0.2919 - val_accuracy: 0.9132 - val_loss: 0.2540
Epoch 7/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9392 - loss: 0.2206 - val_accuracy: 0.9410 - val_loss: 0.2081
Epoch 8/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9406 - loss: 0.2104 - val_accuracy: 0.9375 - 

In [11]:
# 4. 평가
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)
print(f"[{MODEL.upper()}] Test Accuracy: {test_acc:.4f}")

[MLP] Test Accuracy: 0.9806


In [12]:
# 5. 추가 지표 (혼동행렬, 분류 리포트)
y_pred_prob = model.predict(X_test, verbose=0)
y_pred = np.argmax(y_pred_prob, axis=1)
y_true = np.argmax(y_test, axis=1)

print("\nClassification Report")
print(metrics.classification_report(y_true, y_pred, digits=4))

print("Confusion Matrix")
print(metrics.confusion_matrix(y_true, y_pred))


Classification Report
              precision    recall  f1-score   support

           0     1.0000    1.0000    1.0000        36
           1     0.9211    0.9722    0.9459        36
           2     1.0000    1.0000    1.0000        35
           3     1.0000    0.9730    0.9863        37
           4     1.0000    1.0000    1.0000        36
           5     0.9487    1.0000    0.9737        37
           6     1.0000    0.9722    0.9859        36
           7     0.9730    1.0000    0.9863        36
           8     0.9688    0.8857    0.9254        35
           9     1.0000    1.0000    1.0000        36

    accuracy                         0.9806       360
   macro avg     0.9811    0.9803    0.9804       360
weighted avg     0.9811    0.9806    0.9804       360

Confusion Matrix
[[36  0  0  0  0  0  0  0  0  0]
 [ 0 35  0  0  0  1  0  0  0  0]
 [ 0  0 35  0  0  0  0  0  0  0]
 [ 0  0  0 36  0  1  0  0  0  0]
 [ 0  0  0  0 36  0  0  0  0  0]
 [ 0  0  0  0  0 37  0  0  0  0]
 [ 